## Group 4 - Lab Exercise 3

**This file is only used for describing what does the raw data look like and guides us how to process it.**
**It is not intended to be included in the data pipelines**

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Loading the data
df_branch_service = pd.read_json("branch_service_transaction_info.json")
df_customer_transaction = pd.read_json("customer_transaction_info.json")
df_merged = pd.merge(df_customer_transaction, df_branch_service)

# **Data Profiling and Validation**


## Looking at the Raw Data


using .sample() for randomized lookup

In [3]:
df_branch_service.sample(10)

,txn_id,branch_name,service,price
109981,TXN-28049,Starmall,FootSpa,100.121230
122197,TXN-41430,SmallMall,Pedicure,77.987989
5314,TXN-17556,SmallMall,HairColor,NaN
58252,TXN-02258,Starmall,FootSpa,100.121230
121794,TXN-43458,RobinsonsMall,Manicure,55.232400
16894,TXN-03818,None,HairColor,NaN
2125,TXN-23734,RobinsonsMall,Pedicure,NaN
20871,TXN-44137,MayMall,Manicure,0.000000
81502,TXN-14098,MallOfAsia,Pedicure,77.987989
94531,TXN-16795,Megamall,Manicure,55.232400


In [4]:
df_customer_transaction.sample(10)

,txn_id,avail_date,last_name,first_name,birthday
16124,TXN-32266,2017-10-10,BERNHARD,ZION,2001-06-15
20802,TXN-64462,2028-04-28,dietrich,amelie,1992-05-09
90676,TXN-23097,2028-01-04,Stracke,Daphne,2008-08-30
54354,TXN-50431,2028-04-07,Volkman,Ophelia,1999-03-31
6418,TXN-44586,2022-07-22,HOMENICK,SIGMUND,2007-01-13
76595,TXN-03133,2012-11-13,"Hartmann,,,,,,,,",Veronica........,2004-04-20
71844,TXN-50202,2018-11-19,"Miller,,,,,,,,",Haylee........,1993-08-19
42165,TXN-53512,2005-04-04,Wintheiser,Lazaro,2006-08-30
22404,TXN-60238,2022-10-21,rowe,hanna,1998-01-22
61595,TXN-24232,2023-09-14,"Jacobi,,,,,,,,",Camille........,1992-10-02


## Checking for Duplicates

.shape to get row and column count

In [5]:
print(df_branch_service.shape)
print(df_customer_transaction.shape)

(130653, 4)
(130653, 5)


to count unique txn_id, use .nunique()

In [6]:
print(df_branch_service['txn_id'].nunique())
print(df_customer_transaction['txn_id'].nunique())

62354
62354


difference between the original and unique txn_id row count means that there are **duplicate rows**

## Checking for Empty Values

checking null values count per column

In [7]:
df_branch_service.isnull().sum()

txn_id             0
branch_name    10308
service            0
price          20000
dtype: int64

In [8]:
df_customer_transaction.isnull().sum()

txn_id        0
avail_date    0
last_name     0
first_name    0
birthday      0
dtype: int64

branch_name and price have null values

### Empty Values of branch_name Column

look at the possible values of branch_name

In [9]:
df_branch_service['branch_name'].unique()

array(['MallOfAsia', 'Starmall', 'SmallMall', 'MayMall', 'FrankMall',
       'Megamall', 'RobinsonsMall', '', None, 'N/A'], dtype=object)

two types of empty value in the branch_name column: '' & None ('N/A' not considered as empty since it has a different meaning)

Let '' also be null so that we can use one fill method for both of the possible empty values

Can fill the empty values up via forward fill or backward fill

In [10]:
df_branch_service['branch_name'] = df_branch_service['branch_name'].ffill().bfill()

### Empty Values of price Column

since there are empty values on a numerical column, it is best to mean fill its empty values

look at the price mean, the group mean from service, and the group mean from branch_name & service

In [11]:
df_branch_service['price'].describe()

count    110653.000000
mean         95.966969
std         115.874117
min           0.000000
25%          30.123790
50%          66.123457
75%          88.093930
max         400.231230
Name: price, dtype: float64

In [12]:
df_branch_service.groupby(['service'])['price'].describe()

,count,mean,std,min,25%,50%,75%,max
service,,,,,,,,
FootSpa,15835.0,82.240407,38.348651,0.0,100.121230,100.121230,100.121230,100.121230
HairColor,15768.0,72.143386,33.923432,0.0,88.093930,88.093930,88.093930,88.093930
Haircut,15394.0,54.306799,25.333112,0.0,66.123457,66.123457,66.123457,66.123457
Manicure,15676.0,45.275347,21.232941,0.0,55.232400,55.232400,55.232400,55.232400
NailColor,16207.0,24.369285,11.842374,0.0,30.123790,30.123790,30.123790,30.123790
Pedicure,15830.0,64.503900,29.492902,0.0,77.987989,77.987989,77.987989,77.987989
Rebond,15943.0,327.254238,154.542965,0.0,400.231230,400.231230,400.231230,400.231230


In [13]:
df_branch_service.groupby(['branch_name', 'service'])['price'].describe()

count        mean         std  min         25%  \
branch_name service                                                      
            FootSpa      56.0   82.242439   38.692762  0.0  100.121230   
            HairColor    48.0   71.576318   34.748015  0.0   88.093930   
            Haircut    1786.0   53.239379   26.197797  0.0   66.123457   
            Manicure     66.0   49.374418   17.137217  0.0   55.232400   
            Pedicure     57.0   67.042306   27.329980  0.0   77.987989   
...                       ...         ...         ...  ...         ...   
Starmall    Haircut    1859.0   54.243288   25.392248  0.0   66.123457   
            Manicure   2260.0   45.456754   21.084730  0.0   55.232400   
            NailColor  1972.0   24.257900   11.931738  0.0   30.123790   
            Pedicure   2209.0   64.431000   29.561562  0.0   77.987989   
            Rebond     2262.0  328.040982  153.921518  0.0  400.231230   

                              50%         75%         max  
branch_name service                                        
            FootSpa    100.121230  100.121230  100.121230  
            HairColor   88.093930   88.093930   88.093930  
            Haircut     66.123457   66.123457   66.123457  
            Manicure    55.232400   55.232400   55.232400  
            Pedicure    77.987989   77.987989   77.987989  
...                           ...         ...         ...  
Starmall    Haircut     66.123457   66.123457   66.123457  
            Manicure    55.232400   55.232400   55.232400  
            NailColor   30.123790   30.123790   30.123790  
            Pedicure    77.987989   77.987989   77.987989  
            Rebond     400.231230  400.231230  400.231230  

[61 rows x 8 columns]

it is best to use the group mean from branch_name and service to accurately fill up null values in the specific group

## Standardizing Data

### Dates

In [14]:
print(df_merged['avail_date'].describe)
print(df_merged['birthday'].describe)

<bound method NDFrame.describe of 0         2030-09-08
1         2030-09-08
2         2012-04-03
3         2012-04-03
4         2026-05-26
             ...    
404182    2025-01-05
404183    2025-01-05
404184    2025-01-05
404185    2025-01-05
404186    2025-01-05
Name: avail_date, Length: 404187, dtype: object>
<bound method NDFrame.describe of 0         1990-07-08
1         1990-07-08
2         1999-04-08
3         1999-04-08
4         2000-11-26
             ...    
404182    1993-01-10
404183    1993-01-10
404184    1993-01-10
404185    1993-01-10
404186    1993-01-10
Name: birthday, Length: 404187, dtype: object>


currently the two dates in our data is in object data type, convert it to datetime data type

In [15]:
df_merged['avail_date'] = pd.to_datetime(df_merged['avail_date'], format='%Y-%m-%d')
df_merged['birthday'] = pd.to_datetime(df_merged['birthday'], format='%Y-%m-%d')

There are certain rows that are **impossible** to happen looking at our dates columns:

In [16]:
print(df_merged['avail_date'].max())

2030-12-30 00:00:00


dates that are **later than the current date** are invalid and should be removed

In [17]:
from datetime import date
today = str(date.today())
df_merged = df_merged[(df_merged['avail_date'] <= today) | (df_merged['birthday'] <= today)]

avail_date that happens **before** the birthday are also invalid and should be removed

In [18]:
df_merged = df_merged[(df_merged['avail_date'] > df_merged['birthday'])]

### first_name & last_name

The name columns have values that contains special characters. Values in it also does not stick with one format

In [19]:
df_merged.sample(20)

,txn_id,avail_date,last_name,first_name,birthday,branch_name,service,price
281756,TXN-52248,2018-03-25,"Bergnaum,,,,,,,,",Janie........,2009-02-14,MayMall,Rebond,400.231230
274693,TXN-43825,2028-06-13,Mraz,Jose,1991-11-30,Starmall,HairColor,88.093930
262837,TXN-59575,2014-01-25,Kutch,Carissa,1992-02-16,MayMall,HairColor,88.093930
87939,TXN-18122,2024-04-04,WALKER,DOMENICK,2007-05-30,RobinsonsMall,HairColor,NaN
48572,TXN-60191,2005-09-10,Cummings,Shanelle,2003-07-18,Megamall,HairColor,NaN
296052,TXN-25322,2014-06-12,"Jerde,,,,,,,,",Jamil........,1990-05-28,Megamall,Pedicure,77.987989
160137,TXN-05547,2023-04-26,HESSEL,JUNIOR,2004-10-06,FrankMall,FootSpa,NaN
12537,TXN-32409,2020-11-15,GLEICHNER,KATHRYNE,1992-07-15,RobinsonsMall,Haircut,NaN
196402,TXN-41803,2007-12-19,mosciski,rae,2004-01-21,FrankMall,Haircut,0.000000
137705,TXN-51440,2024-12-23,Schuster,Maegan,2002-12-23,Starmall,FootSpa,100.121230


only allow letters in this columns and stick to one format (e.g. all uppercase)